<a href="https://www.kaggle.com/code/achopra20/baseline-yolo-model?scriptVersionId=283475437" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Using YOLO Model

## Load required libraries 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

## List all input files

In [ ]:
# include dataset, videos and images that require for testing

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Check dataset type and annotation types

In [ ]:
import os, glob, json, xml.etree.ElementTree as ET

ROOT = "/kaggle/input/pothole-detection-dataset"  # change to your slug
print("Some files:", sorted(os.listdir(ROOT))[:20])

# Check common annotation types
xmls = glob.glob(f"{ROOT}/**/*.xml", recursive=True)
txts = glob.glob(f"{ROOT}/**/*.txt", recursive=True)
jsons = glob.glob(f"{ROOT}/**/*.json", recursive=True)

print("XML count:", len(xmls), "TXT count:", len(txts), "JSON count:", len(jsons))


## Install YOLO framework

In [ ]:
!pip install -q ultralytics

## Load YOLO base model for fast processing

In [ ]:
from ultralytics import YOLO
baseline_model = YOLO('yolov8n.yaml')   # start with n or s; n is lighter - Baseline model (no pretrained)

In [ ]:
yaml_content = """
path: /kaggle/input/pothole-detection-dataset/MWPDD

train: train/images
val: valid/images
test: test/images

nc: 1
names: ['pothole']
"""

with open("pothole.yaml", "w") as f:
    f.write(yaml_content)

print("✅ pothole.yaml created successfully")


## Train YOLO model based on dataset

In [ ]:

baseline_result = baseline_model.train(
                                        data="pothole.yaml",
                                        epochs=50,
                                        imgsz=640,
                                        batch=16,
                                        lr0=1e-3,
                                        project="pothole_yolo",
                                        name="baseline_scratch",
                                        pretrained=False
                                    )

## Evaluate baseline model

In [ ]:
baseline_metrics = baseline_model.val(data="pothole.yaml")
print(baseline_metrics)

## Find saved trained model directory

In [ ]:
import os
print(baseline_model.trainer.save_dir)

## Load trained model and validate on test dataset

In [ ]:
best = YOLO('/kaggle/working/pothole_yolo/baseline_scratch2/weights/best.pt')

#best val is used for result comparision with ground truth, it uses test data to comparision with trained dataset
metrics = best.val(data="pothole.yaml", split="test", imgsz=640)
metrics.box.map50, metrics.box.map

## Display Confusion Matrix

In [ ]:
from IPython.display import Image, display

display(Image(filename="/kaggle/working/runs/detect/val2/confusion_matrix.png"))

## Find Accuracy based on result

In [ ]:
cm = metrics.confusion_matrix.matrix
TP, FP = cm[0]
FN, TN = cm[1]

accuracy = (TP + TN) / (TP + TN + FP + FN)
print("Accuracy:", accuracy)

## Use real video to detect the pothole

In [ ]:
VIDEO_IN  = "/kaggle/input/road-surface/road_surface.mp4"  # input video
VIDEO_OUT = "/kaggle/working/pothole_annotated.mp4" # output saved video

best = YOLO('/kaggle/working/pothole_yolo/baseline_scratch2/weights/best.pt')
best.predict(source=VIDEO_IN, conf=0.35, save=True, project="/kaggle/working", name="vidruns", imgsz=640)

# Ultralytics saves annotated video inside /kaggle/working/vidruns/
import glob, shutil, os
cand = glob.glob("/kaggle/working/vidruns/*.mp4")
if cand:
    shutil.copy(cand[0], VIDEO_OUT)
    print("Saved:", VIDEO_OUT)